# Imports

In [10]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

from colored import fg, bg, attr

# Load and Explore Data

In [11]:
train = pd.read_csv("Dataset3_train.csv")

## Review Number of features and datapoints in both sets

In [12]:
print(f"Dimensions of training data are:")
print(fg(2), attr(1), f"{train.shape}\n", fg(15), attr(0))

print(f"\nAll the present features in the dataset are:")
print(fg(2), attr(1), f"{list(train.columns)[:-1]}\n", fg(15), attr(0))
print("A quick look at the head of trainset:\n")
train.head()


Dimensions of training data are:
  (45000, 22)
  

All the present features in the dataset are:
  ['id', 'dered_i', 'dered_z', 'dered_u', 'dered_g', 'dered_r', 'err_i', 'err_z', 'err_u', 'err_g', 'err_r', 'extinction_r', 'skyVersion', 'run', 'rerun', 'camCol', 'field', 'obj', 'photoz', '#ra', 'dec']
  
A quick look at the head of trainset:



,id,dered_i,dered_z,dered_u,dered_g,dered_r,err_i,err_z,err_u,err_g,...,skyVersion,run,rerun,camCol,field,obj,photoz,#ra,dec,class
0,0,15.1459,14.9723,18.6394,16.4080,15.4896,0.00195,0.00406,0.02796,0.00279,...,1,745,40,2,518,5,2.64988,236.22077,-0.58795,0
1,1,17.3579,17.3498,18.5273,17.6527,17.4296,0.00757,0.02498,0.02484,0.00536,...,1,745,40,2,518,8,2.63438,236.22135,-0.46320,0
2,2,20.6154,19.7785,25.4019,23.4824,22.5504,0.10071,0.16768,0.75381,0.46380,...,1,745,40,2,518,9,1.65738,236.22055,-0.46434,0
3,3,15.0188,14.9927,16.7971,15.5712,15.1534,0.00183,0.00411,0.00820,0.00195,...,1,745,40,2,518,12,2.28431,236.22224,-0.61553,0
4,4,20.3008,21.6454,23.7314,21.7139,20.9195,0.21194,1.86698,2.73388,0.27544,...,1,745,40,2,518,13,1.34673,236.22307,-0.61710,1


## Checking for Missing Values

In [13]:
print("Checking for missing or null values in the dataset:\n")
print(fg(3), attr(1), train.isnull().values.any())
print(fg(2), attr(1), "No missing values present in train set", fg(15), attr(0))

Checking for missing or null values in the dataset:

  False
  No missing values present in train set  


## Check datatype of features

In [14]:
print(train.info())
print(fg(2), attr(1), "All values in dataset are numerical, no categorical or nominal variables, thus no need for one-hot encoding", fg(15), attr(0))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            45000 non-null  int64  
 1   dered_i       45000 non-null  float64
 2   dered_z       45000 non-null  float64
 3   dered_u       45000 non-null  float64
 4   dered_g       45000 non-null  float64
 5   dered_r       45000 non-null  float64
 6   err_i         45000 non-null  float64
 7   err_z         45000 non-null  float64
 8   err_u         45000 non-null  float64
 9   err_g         45000 non-null  float64
 10  err_r         45000 non-null  float64
 11  extinction_r  45000 non-null  float64
 12  skyVersion    45000 non-null  int64  
 13  run           45000 non-null  int64  
 14  rerun         45000 non-null  int64  
 15  camCol        45000 non-null  int64  
 16  field         45000 non-null  int64  
 17  obj           45000 non-null  int64  
 18  photoz        45000 non-nu

## Check distribution of data in different columns

In [15]:
fig, axes = plt.subplots(5, 4)
fig.set_figheight(150)
fig.set_figwidth(120)
columns = train.columns[1:-1]
for i in range(5):
    for j in range(4):
        axes[i, j].hist(train[columns[4*i +j]])
        axes[i,j].set_title(columns[4*i + j])

In [16]:
train = train.drop(['err_i','err_z','err_u','err_g','err_r','skyVersion','rerun','id'], axis=1)
train = train.rename(columns={"#ra": "ra", "class": "aclass"})
print(attr(1), fg(2), "Columns having the same value for all datapoints, i.e., redundant columns have been dropped!", fg(15), attr(0))

  Columns having the same value for all datapoints, i.e., redundant columns have been dropped!  


## Separate data into features and labels

In [17]:
X = train.drop(["aclass"], axis=1)
y = train["aclass"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
X_train.columns

Index(['dered_i', 'dered_z', 'dered_u', 'dered_g', 'dered_r', 'extinction_r',
       'run', 'camCol', 'field', 'obj', 'photoz', 'ra', 'dec'],
      dtype='object')

### A look at the overall distribution of classes for the train and test set

In [19]:
print("Distribution of classes in training set:\n", fg(2), attr(1))
print(y_train.value_counts(), fg(15), attr(0))
print("Distribution of classes in test set:\n", fg(2), attr(1))
print(y_test.value_counts(), fg(15), attr(0))

Distribution of classes in training set:
  
0    15594
1    10803
2     9603
Name: aclass, dtype: int64  
Distribution of classes in test set:
  
0    3888
1    2649
2    2463
Name: aclass, dtype: int64  


# Create Pipeline of Models to be trained

In [20]:
names = ['LogisticRegression', 'RandomForest', 'AdaBoost','GaussianNB','MLP','GradientBoosting', 'SVC', 'XGBoost']
classifiers = [LogisticRegression(), RandomForestClassifier(class_weight='balanced'), AdaBoostClassifier(), GaussianNB(), MLPClassifier(), GradientBoostingClassifier(), OneVsRestClassifier(SVC()), XGBClassifier()]

# Experiments

## Normal Run

### Train Models

In [21]:
for name, clf in zip(names, classifiers):
    print(f"Training {name}")
    clf.fit(X=X_train, y=y_train)
    print(f"Training accuracy for {name}: {clf.score(X=X_train, y=y_train)}")

Training LogisticRegression
Training accuracy for LogisticRegression: 0.5692222222222222
Training RandomForest
Training accuracy for RandomForest: 1.0
Training AdaBoost
Training accuracy for AdaBoost: 0.5947222222222223
Training GaussianNB
Training accuracy for GaussianNB: 0.5099444444444444
Training MLP
Training accuracy for MLP: 0.5246944444444445
Training GradientBoosting
Training accuracy for GradientBoosting: 0.6641388888888888
Training SVC
Training accuracy for SVC: 0.4817222222222222
Training XGBoost
Training accuracy for XGBoost: 0.7938055555555555


### Test Models

In [22]:
for name, clf in zip(names, classifiers):
    preds = clf.predict(X_test)
    score = accuracy_score(preds, y_test)
    print(f"Testing Accuracy of {name} is {score}")

Testing Accuracy of LogisticRegression is 0.5693333333333334
Testing Accuracy of RandomForest is 0.6667777777777778
Testing Accuracy of AdaBoost is 0.5908888888888889
Testing Accuracy of GaussianNB is 0.5098888888888888
Testing Accuracy of MLP is 0.5236666666666666
Testing Accuracy of GradientBoosting is 0.6398888888888888
Testing Accuracy of SVC is 0.4816666666666667
Testing Accuracy of XGBoost is 0.6634444444444444


### Save Models

In [26]:
for name, clf in zip(names, classifiers):
    pickle.dump(clf, open(f"{name}", 'wb+'))

## GridSearchCV for the 2 Best models

### RandomForestClassifier

In [24]:
parameters = {'n_estimators': [200, 400, 600], 'bootstrap': [True, False], 'max_depth': [70, 80,None], 'max_features': ['auto','sqrt']}
RandomForestCV = GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced'), param_grid=parameters, n_jobs=-1, verbose=3)

In [25]:
RandomForestCV.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [67]:
accuracy_score(RandomForestCV.predict(X_test), y_test)

0.6727777777777778

In [68]:
pickle.dump(RandomForestCV, open("RandomForestCV", "wb+"))

### GradientBoostingClassifier

In [78]:
parameters = {'n_estimators': [200, 400, 600], 'loss': ['deviance', 'exponential'], 'criterion': ['friedman_mse', 'mse', 'mae'], 'max_features': ['auto','sqrt', 'log2'], 'learning_rate':[0.05, 0.1, 0.2]}
GradientBoostingCV = GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=parameters, n_jobs=-1, verbose=3)

In [79]:
GradientBoostingCV.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 13.9min


KeyboardInterrupt: 

In [82]:
parameters = {'min_child_weight': [1, 5, 10],'gamma': [0.5, 1, 1.5, 2, 5],'subsample': [0.6, 0.8, 1.0],                         'colsample_bytree': [0.6, 0.8, 1.0],'max_depth': [3, 4, 5]}
XGBoostCV = GridSearchCV(estimator=XGBClassifier(), param_grid=parameters, n_jobs=-1, verbose=3)
XGBoostCV.fit(X_train, y_train)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 